# README.
This notebook synchronizes the database with the CSV file in whhdata.

In [1]:
import os
import glob
import pandas as pd
import dbutils

/data/anaconda/envs/datathon/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


# Locate and load CSV from whhdata.

In [2]:
# Where to get the data.
whhdata_path = "/whhdata"
glob_search_path = os.path.join(whhdata_path, "*.csv")
csv_paths = sorted(glob.glob(glob_search_path))
csv_path = csv_paths[0]
print("Using {}".format(csv_paths))

Using ['/whhdata/results-measures-20190319-102143.csv']


In [3]:
# Load the data-frame.
df = pd.read_csv(csv_path)

In [4]:
df.head()

,id,personId,qrcode,sex,type,age,height,weight,muac,headCircumference,oedema,latitude,longitude,address,timestamp,deleted,deletedBy,visible,createdBy
0,fb3bfc1a748e12ee_measure_1552552380603_hfPIPRf...,fb3bfc1a748e12ee_mandloi_1552552136224_j441qmO...,NaN,NaN,manual,1371,90.5,10.70,13.5,46.2,False,22.337960,74.953553,NaN,1552890976734,False,NaN,False,cgm15whh@gmail.com
1,fb3bfc1a748e12ee_measure_1552378132366_b3zadc7...,fb3bfc1a748e12ee_jamra_1552378019288_CilAQntVd...,NaN,NaN,manual,1104,85.4,9.75,14.1,43.3,False,22.363275,74.999870,NaN,1552891706555,False,NaN,False,cgm15whh@gmail.com
2,fb3bfc1a748e12ee_measure_1552374894278_oLUZcV3...,fb3bfc1a748e12ee_nigwal_1552374875042_YyE4As6U...,NaN,NaN,manual,617,76.3,8.20,13.5,45.0,False,22.363275,74.999870,NaN,1552891706483,False,NaN,False,cgm15whh@gmail.com
3,fb3bfc1a748e12ee_measure_1552376998993_qhSkTpX...,fb3bfc1a748e12ee_jamra_1552376878360_GZ2RfdJKN...,NaN,NaN,manual,707,74.0,7.30,11.6,44.3,False,22.363275,74.999870,NaN,1552891706532,False,NaN,False,cgm15whh@gmail.com
4,fb3bfc1a748e12ee_measure_1552376299613_pDjxYHV...,fb3bfc1a748e12ee_buawar_1552376170986_vFRpgUMz...,NaN,NaN,manual,608,71.0,7.65,12.6,42.6,False,22.363275,74.999870,NaN,1552891706509,False,NaN,False,cgm15whh@gmail.com


In [5]:
columns = list(df)
columns

['id',
 'personId',
 'qrcode',
 'sex',
 'type',
 'age',
 'height',
 'weight',
 'muac',
 'headCircumference',
 'oedema',
 'latitude',
 'longitude',
 'address',
 'timestamp',
 'deleted',
 'deletedBy',
 'visible',
 'createdBy']

In [6]:
columns_mapping = { column: column for column in columns}
columns_mapping["id"] = "measurement_id"
columns_mapping["personId"] = "person_id"
columns_mapping["age"] = "age_days"
columns_mapping["height"] = "height_cms"
columns_mapping["weight"] = "weight_cms"
columns_mapping["muac"] = "muac_cms"
columns_mapping["headCircumference"] = "head_circumference_cms"
columns_mapping["deletedBy"] = "deleted_by"
columns_mapping["createdBy"] = "created_by"
columns_mapping["personId"] = "person_id"
columns_mapping

{'id': 'measurement_id',
 'personId': 'person_id',
 'qrcode': 'qrcode',
 'sex': 'sex',
 'type': 'type',
 'age': 'age_days',
 'height': 'height_cms',
 'weight': 'weight_cms',
 'muac': 'muac_cms',
 'headCircumference': 'head_circumference_cms',
 'oedema': 'oedema',
 'latitude': 'latitude',
 'longitude': 'longitude',
 'address': 'address',
 'timestamp': 'timestamp',
 'deleted': 'deleted',
 'deletedBy': 'deleted_by',
 'visible': 'visible',
 'createdBy': 'created_by'}

In [17]:
json_data = dbutils.load_dbconnection_file()

connection = dbutils.connect_to_main_database()
cursor = connection.cursor()

cursor.execute("SELECT table_name FROM information_schema.tables WHERE table_schema = 'public'")
for table in cursor.fetchall():
    print(table)
    
result = cursor.execute("""SELECT datname from pg_database""")
print(result)

result = cursor.execute("SELECT current_database();")
print(result)



for index, row in df.iterrows():
    sql_statement = "INSERT INTO {}".format("measurements") + " "
    
    keys = []
    values = []
    for df_key, db_key in columns_mapping.items():
        keys.append(str(db_key))
        values.append("'" + str(row[df_key]) + "'")
    
    keys_string = "(" + ", ".join(keys) + ")"
    values_string = "VALUES (" + ", ".join(values) + ")"
    sql_statement += keys_string
    sql_statement += "\n" + values_string
    sql_statement += ";"
    print(sql_statement)
    
    result = cursor.execute(sql_statement)
    print(result)
    
    result = cursor.execute("SELECT * from measurements;")
    print(result)

    break



connection.commit()

('measurements',)
None
None
INSERT INTO measurements (measurement_id, person_id, qrcode, sex, type, age_days, height_cms, weight_cms, muac_cms, head_circumference_cms, oedema, latitude, longitude, address, timestamp, deleted, deleted_by, visible, created_by)
VALUES ('fb3bfc1a748e12ee_measure_1552552380603_hfPIPRfiBlsY6ioL', 'fb3bfc1a748e12ee_mandloi_1552552136224_j441qmOri05Mt5bm', 'nan', 'nan', 'manual', '1371', '90.5', '10.7', '13.5', '46.2', 'False', '22.33795972', '74.95355289', 'nan', '1552890976734', 'False', 'nan', 'False', 'cgm15whh@gmail.com');
None
None


In [18]:
result = cursor.execute("SELECT * from measurements;")
cursor.fetchall()



[(1,
  'fb3bfc1a748e12ee_measure_1552552380603_hfPIPRfiBlsY6ioL',
  'fb3bfc1a748e12ee_mandloi_1552552136224_j441qmOri05Mt5bm',
  'nan',
  'nan',
  'manual',
  1371,
  90.5,
  10.7,
  13.5,
  46.2,
  False,
  22.338,
  74.9536,
  'nan',
  Decimal('1552890976734'),
  False,
  'nan',
  False,
  'cgm15whh@gmail.com'),
 (2,
  'fb3bfc1a748e12ee_measure_1552552380603_hfPIPRfiBlsY6ioL',
  'fb3bfc1a748e12ee_mandloi_1552552136224_j441qmOri05Mt5bm',
  'nan',
  'nan',
  'manual',
  1371,
  90.5,
  10.7,
  13.5,
  46.2,
  False,
  22.338,
  74.9536,
  'nan',
  Decimal('1552890976734'),
  False,
  'nan',
  False,
  'cgm15whh@gmail.com'),
 (3,
  'fb3bfc1a748e12ee_measure_1552552380603_hfPIPRfiBlsY6ioL',
  'fb3bfc1a748e12ee_mandloi_1552552136224_j441qmOri05Mt5bm',
  'nan',
  'nan',
  'manual',
  1371,
  90.5,
  10.7,
  13.5,
  46.2,
  False,
  22.338,
  74.9536,
  'nan',
  Decimal('1552890976734'),
  False,
  'nan',
  False,
  'cgm15whh@gmail.com'),
 (4,
  'fb3bfc1a748e12ee_measure_1552552380603_hfPIP